In [75]:
import pandas as pd
import spacy
import torch
from transformers import BertTokenizer
from torchtext import data
import numpy as np
import random

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [76]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [77]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
print(max_input_length)

512


In [78]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [79]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [80]:
USER_REVIEW = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

USER_SUGGESTION = data.Field(sequential=False, unk_token=None)

fields = {'user_review': ('ur', USER_REVIEW), 'user_suggestion': ('us', USER_SUGGESTION)}

In [81]:
train_data, test_data = data.TabularDataset.splits(path = 'latest_cleaned_data',train = 'train.csv',
                                                   test = 'test_from_train.csv',format = 'csv',
                                                   fields = fields)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [87]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 8572
Number of validation examples: 3673
Number of testing examples: 5249


In [83]:
print(vars(train_data.examples[6]))

{'ur': [1016, 2243, 16068, 2145, 2302, 1999, 2208, 11403, 1017, 2570, 2052, 2209, 4183, 2062, 2084, 2095, 3283, 1045, 2404, 1037, 9467, 2006, 2023, 2208, 2022, 27199, 2063, 2009, 3335, 1037, 2489, 6834, 11403, 5685, 2017, 2113, 2054, 2009, 2022, 2145, 3335, 2021, 2145, 1017, 2570, 2052, 2377], 'us': 'neg'}


In [84]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[1])['ur'])

print(tokens)

['an', 'explosion', 'go', 'off', 'in', 'my', 'house', 'halfway', 'through', 'a', 'game', 'of', 'conquest', 'with', 'me', 'play', 'ah', 'pu', '##ch', 'someone', 'who', 'blow', 'stuff', 'up', 'still', 'finish', 'the', 'game', 'turn', 'out', 'it', 'be', 'a', 'coconut', 'that', 'explode', 'in', 'half', 'my', 'mind', 'be', 'blow', '10', '10', 'dedication', 'fun', 'game']


In [85]:
USER_SUGGESTION.build_vocab(train_data)

In [86]:
print(USER_SUGGESTION.vocab.stoi)

defaultdict(None, {'pos': 0, 'neg': 1})


In [89]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [90]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [91]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [92]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [93]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


In [94]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [95]:
criterion = nn.BCEWithLogitsLoss()

In [96]:
model = model.to(device)
criterion = criterion.to(device)

In [106]:

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [115]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.ur).squeeze(1)
        
        loss = criterion(predictions, batch.us)
        
        acc = binary_accuracy(predictions, batch.us)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [116]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.ur).squeeze(1)
            
            loss = criterion(predictions, batch.us)
            
            acc = binary_accuracy(predictions, batch.us)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [117]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [118]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 1.96 GiB total capacity; 910.99 MiB already allocated; 12.38 MiB free; 954.00 MiB reserved in total by PyTorch)

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [30]:
train = pd.read_csv("Kaggle_Real/train.csv")
train = train.loc[:,['user_review','user_suggestion']]

In [31]:
X_train,X_test,y_train,y_test = train_test_split(train.user_review.values,train.user_suggestion.values,stratify=train.user_suggestion.values,test_size=0.01)
train_dict = {'user_review':X_train,'user_suggestion':y_train}
train_json = pd.DataFrame(train_dict)
test_dict = {'user_review':X_test,'user_suggestion':y_test}
test_json = pd.DataFrame(test_dict)
train_json.loc[train_json['user_suggestion'] == 1,'user_suggestion'] = 'pos'
train_json.loc[train_json['user_suggestion'] == 0,'user_suggestion'] = 'neg'
test_json.loc[test_json['user_suggestion'] == 1,'user_suggestion'] = 'pos'
test_json.loc[test_json['user_suggestion'] == 0,'user_suggestion'] = 'neg'
train_json.to_csv("bert_train.csv",index=False)
test_json.to_csv("bert_val.csv",index=False)